# Problem Set 3 (Splay Tree)
This notebook contains a Python Implementation of a Splay Tree.

The difference between such a tree and a normal binary search tree is that after adding or searching a node, the node is splayed to the top of the tree, and after deleting a node, its parent is splayed to the top of the tree. This characteristic of recently-accessed nodes being near the top of the tree means that frequently accessed nodes can be searched very quickly, making it superior to normal search trees when used with non-random datasets.

## Program Design

Tasks:
1. Addition of movie records
2. Deletion of movie records
3. Search by movie title (full title)
4. Count number of titles with less than <em>y</em> million dollars in <code>box office</code>
5. Displays all titles with <code>box office</code> within range <em>[a, b]</em>
6. Printing the movie that has the highest <code>box office</code>
7. Printing of movie titles sorted by ascending <code>score</code>
8. Displays all titles with more than <em>x</em> <code>score</code>

## Time Complexity

Unlike binary search trees, which have a worst-case time complexity of O(log n) for access, insertion, and deletion, splay trees have worst-case operations of O(n), although these operations are infrequent, so its amortized time complexity is still O(log n).

However, the tradeoff that this provides is extremely fast access to more frequently accessed nodes. With movie datasets, where the movie with the higher scores and popularity will get searched much more than the others, Tasks 7 and 8 might be finished in O(1) time.

Furthermore, the splaying operations performed give the tree self-balancing properties, allowing for faster average search times than normal search trees.

## Space Complexity

This implementation uses linked nodes, making it more memory intensive than single arrays. However, some measures have been taken to prevent memory usage from being too much.

Firstly, the class attributes for nodes and trees were stored in static amounts of memory allocated for the fixed number of attributes instead of the dynamic dictionary that Python defaults to using. This simple one-line settings can reduce RAM usage by up to 40%.

Secondly, deleted nodes are cleared from memory using the Python garbage collector function, ensuring there is no wasted memory when nodes are deleted.

With these measures, the tree and nodes will use up much less memory, so it will not be a huge problem.

## Other Stuff

__1. Update values__
With movies, the reviews and scores can constantly change, so being able to update the values in a tree are important.
In this implementation, values can easily be updated with the SplayBST.add(key, newvalue) function.

__2. Length__
Without an attribute in the tree to keep track of the number of nodes in an array, the only way to measure length would be to do a full traversal in O(n) time. By keeping track, this tree can return its length in O(n) time.

__3. Many Traversals__
The tree also supports many ways to traverse and return the tree: preorder, postorder inorder, reverse inorder, and level order. This can be useful when debugging or building tree diagrams. (Hopefully there aren't any bugs though)

## Downloading movie data, import dependencies

In [1]:
!rm movies.txt
!wget 'https://gist.githubusercontent.com/lorraineneo/01cddfb978e3a243e18107c21b476e65/raw/7871f592d4f7435fcfca095f9c53ee32d78bdfc3/movies.txt' --quiet --no-check-certificate
import gc

In [2]:
class Node:
    '''Binary node that contains a key, a value, and left and right pointers to future children'''
    #allocate static amount of memory to variables instead of a dynamic dictionary
    #this can reduce up to 40% of RAM
    __slots__ = ('key', 'val', 'left', 'right')
    def __init__(self, key, value):
        self.key = key
        self.val = value
        self.left = None
        self.right = None

    def __str__(self):
        return str((self.key, self.val))
    
    def __getitem___(self, index):
        if index == 0: return self.key
        if index == 1: return self.val
        
    __repr__ = __str__

In [3]:
class SplayBST:
    '''
    Binary splay tree, where a node is splayed to the top of the tree when it is added to the tree or accessed
    through a binary search. When a node is deleted, the parent of the deleted node is splayed instead.

    Keeping the more frequently accessed nodes near the top of the tree allows for extremely fast access speeds
    when accessing the same few nodes over and over, resulting in possibly O(1) search time complexity.

    Nodes are created with key-value pairs, the value being returned when a node with the corresponding key is found.
    While duplicate key are not allowed, the value of an existing key can be overwritten with a new value.

    To reduce the memory burden that the linked nodes cause, the class instructs Python to only use a static amount
    of memory for our fixed number of variables, reducing RAM usage drastically
    '''

    #allocate static amount of memory to variables instead of a dynamic dictionary
    #this can reduce up to 40% of RAM
    __slots__ = ('root', 'length')

    def __init__(self, key, value):
        '''
        Initializes root node with the given key and value, and starts the length of the tree at 1

        Time complexity: O(1)

        E.g:
        >>> a = SplayBST(1, 2)
        >>> print(a.root)
        (1, 2)
        >>> print(a.length)
        1
        '''
        assert value is not None, "Value cannot be None"
        self.root = Node(key, value)
        self.length = 1

    def __len__(self):
        '''
        Returns length of tree when len() is called

        Time complexity: O(1)

        E.g:
        >>> a = SplayBST(1, 2)
        >>> print(len(a))
        1
        '''
        return self.length

    def __str__(self):
        '''
        Returns string of all nodes in pre-order

        Time complexity: O(1)

        E.g:
        >>> a = SplayBST(1, 2)
        >>> a.add(2, 1)
        >>> a.add(1.5, 3)
        >>> print(a)
        [(1.5, 3), (1, 2), (2, 1)]
        '''
        string = '['
        for i in self._preOrderGen(self.root): string += str(i) + ", "
        return string[:-2] + ']'

    def __iter__(self):
        '''
        Iterates through list of all nodes in ascending order

        Time complexity: O(n^2)

        E.g:
        >>> a = SplayBST(1, 2)
        >>> a.add(2, 1)
        >>> a.add(1.5, 3)
        >>> for i in a:
        >>>     print(i)
        (1, 2)
        (1.5, 3)
        (2, 1)
        '''
        return iter(self._inOrderGen(self.root))

    def __reversed__(self):
        '''
        Returns a list of all nodes in descending order

        Time complexity: O(n)

        >>> a = SplayBST(1, 2)
        >>> a.add(2, 1)
        >>> a.add(1.5, 3)
        >>> b = reversed(a)
        >>> print(b)
        [(2, 1), (1.5, 3), (1, 2)]
        >>> print(type(b))
        list
        '''
        return self._inOrderReverse(self.root)

    def __contains__(self, key):
        '''
        Returns whether key is present in the splay tree

        Time complexity(Average): O(log n)
        Time complexity(Amortized): O(log n)
        Time complexity(Worst-case): O(n)

        E.g:
        >>> a = SplayBST(1, 2)
        >>> print(1 in a)
        True
        >>> print(2 in a)
        False
        '''
        return self.search(key) is not None

    def __setitem__(key, value):
        '''Same as self.add(key, value):''' + self.add.__doc__
        return self.add(key, value)

    def __getitem__(self, key):
        '''Same as self.search(key, value):''' + self.search.__doc__
        return self.search(key)

    def __delitem__(key):
        '''Same as self.delete(key, value):''' + self.delete.__doc__
        return self.delete(key)

    __repr__ = __str__

    def add(self, key, value):
        '''
        Adds a node with the given key and value to the tree and splays it to the top of the tree

        Time complexity(Average): O(log n)
        Time complexity(Amortized): O(log n)
        Time complexity(Worst-case): O(n)

        >>> a = SplayBST(1, 2)
        >>> a.add(2, 3)
        >>> print(a)
        [(2, 3), (1, 2)]
        >>> a.add(1.5, 0)
        [(1.5, 0), (1, 2), (2, 3)]
        '''
        assert value is not None, "Value cannot be None"
        self.root = self.splayNode(self.root, self.addNode(self.root, key, value))

    def search(self, key):
        '''
        Searches for the node with the given key and returns its corresponding value
        If the node does not exist, None is returned. If it does exist, it is splayed to the top of the tree

        Time complexity(Average): O(log n)
        Time complexity(Amortized): O(log n)
        Time complexity(Worst-case): O(n)

        >>> a = SplayBST(1, 2)
        >>> print(a.search(1))
        2
        >>> print(a.search(2))
        None

        '''
        node = self.searchNode(self.root, key)
        self.root = self.splayNode(self.root, node)
        return node.val

    def delete(self, key, pop=False):
        '''
        Searches for the node and removes it from the tree. If pop is set as True, the value of the node is also returned.
        If the node is not present in the tree and pop is True however, None is returned.
        Then, the parent of the deleted node is splayed to the top of the tree.

        Time complexity(Average): O(log n)
        Time complexity(Amortized): O(log n)
        Time complexity(Worst-case): O(n)

        >>> a = SplayBST(1, 2)
        >>> a.add(2, 3)
        >>> a.add(1.5, 0)
        >>> print(a)
        [(1.5, 0), (1, 2), (2, 3)]
        >>> a.delete(1.5)
        >>> print(a)
        [(2, 3), (1, 2)]
        >>> print(a.delete(1, pop=True))
        2
        '''
        assert self.length > 1, "At least one node must be present in tree"
        deleted = self.delNode(self.root, key)
        self.root = self.splayNode(self.root, deleted[0])
        if pop: return deleted[1]

    def addNode(self, root, key, value):
        '''
        Function that inserts a new node into the tree if a new key is specified, and updates value if an existing key is given

        The position is determined by the key (left if it is less than a node, right if it is more than a node)
        '''
        def _insert(root, node):
            #updates value if new value is given
            if root.key == node.key:
                root.val = node.val
                return root
            #when key is more than node's key, inserts node if right of node is empty, continues moving right otherwise
            else:
                if root.key < node.key:
                    if root.right is None: root.right = node
                    else: return _insert(root.right, node)
            #when key is less than node's key, inserts node if left of node is empty, continues moving left otherwise
                if root.key > node.key:
                    if root.left is None: root.left = node
                    else: return _insert(root.left, node)
                self.length += 1
            #return newly inserted node to be splayed later on
            return node
        return _insert(root, Node(key, value))

    def searchNode(self, root, key):
        '''
        Function that binary searches for the node with the given key based on the key's value

        If the key cannot be found, None is returned
        '''
        def _search(root, key):
            #if node's key matches given key, the node is found and returned
            if root.key == key: return root
            #if they do not match, search continues right if the key is more than node's key
            if root.key < key and root.right is not None: return _search(root.right, key)
            #if they do not match, search continues left if the key is less than node's key
            if root.key > key and root.left is not None: return _search(root.left, key)
            #if the search cannot proceed left/right, it is assumed the node does not exist, and None is returned
            return None
        return _search(root, key)

    def delNode(self, root, key):
        '''
        Function that deletes node, clears the memory it takes up, and replaces it with a successor

        If the key cannot be found, (None, None) is returned
        '''
        def _findMinNode(node):
            #keeps moving left until it finds the leftmost node
            if node.left is not None: return _findMinNode(node.left)
            else: return node
        def _replace(node):
            #finds node with key closest to given node (leftmost of right node)
            successor = _findMinNode(node.right)
            #subs in new key-value
            node.key, node.val = successor.key, successor.val
            #adds 1 since the length will be subtracted by 1 later
            self.length += 1
            #removes successor
            _delete(successor)
        def _delete(node):
            #does not bother if node was not found
            if node is not None:
                #stores parent and value to return later
                parent = self._parent(self.root, node.key)
                value = node.val
                #replaces node with its child if it has 0/1 child and is not root
                if (node.left is None or node.right is None) and parent is not None:
                    if parent.left == node: parent.left = node.left or node.right
                    else: parent.right = node.right or node.right
                    del node
                #replaces node with child if it has 0/1 children and is root
                elif (node.left is None or node.right is None):
                    successor = node.left or node.right
                    node.key, node.val, node.left, node.right = successor.key, successor.val, successor.left, successor.right
                    parent = node
                    del successor
                #replaces node with leftmost of right child if it has 2 children
                else: _replace(node)
                #updates length and returns parent and value
                self.length -= 1
                return parent, value
            return None, None
        #garbage collects 'deleted' nodes
        gc.collect()
        return _delete(self.searchNode(root, key))

    def splayNode(self, root, node):
        '''
        Function that splays the node up the tree until it becomes root while maintaining the priority and order of previously splayed nodes
        '''
        #right rotation(zig)
        def _zig(node, parent, grandnode=None):
            parent.left = node.right
            node.right = parent
            if grandnode is not None:
                if grandnode.left == parent: grandnode.left = node
                if grandnode.right == parent: grandnode.right = node
        #left rotation(zag)
        def _zag(node, parent, grandnode=None):
            parent.right = node.left
            node.left = parent
            if grandnode is not None:
                if grandnode.left == parent: grandnode.left = node
                if grandnode.right == parent: grandnode.right = node
        #no need to splay if node is already root
        if root == node or node == None: return root
        #if node is left child of root, a right rotation (or zig) is performed
        if root.left == node:
            _zig(node, root)
            return node
        #if node is right child of root, a left rotation (or zag) is performed
        if root.right == node:
            _zag(node, root)
            return node
        if root.left is not None:
        #if node is left-left grandchild of root, a zig needs to be performed between the parent and grandparent to maintain order
        #this makes the step zig-zig
            if root.left.left == node:
                temp = root.left
                _zig(root.left, root)
                _zig(node, temp)
                return node
        #if node is left-right grandchild of root, the node needs to zag up(while maintaining a connection to root), and then zig up
        #this step is called zig-zag
            elif root.left.right == node:
                _zag(node, root.left, grandnode=root)
                _zig(node, root)
                return node
        #mirrorred version of zig zig (it's now zag zag)
        if root.right is not None:
            if root.right.right == node:
                temp = root.right
                _zag(root.right, root)
                _zag(node, temp)
                return node
        #mirrored version of zig-zag(zag-zig)
            elif root.right.left == node:
                _zig(node, root.right, grandnode=root)
                _zag(node, root)
                return node
        #if the grandparent of the node is not root, the grandparent and great-grandparent are searched
        #with some steps to make sure the program does not spit out ugly errors when the parent is not found
        p = self._parent(self.root, node.key)
        if p is not None: gp = self._parent(self.root, p.key)
        else: return None
        if gp is not None: gpp = self._parent(self.root, gp.key)
        else: return None
        #moves the node up by two levels based on our 4 cases
        if gp.left is not None:
        #zig-zig
            if gp.left.left == node:
                temp = gp.left
                _zig(gp.left, gp, grandnode=gpp)
                _zig(node, temp, grandnode=gpp)
        #zig-zag
            elif gp.left.right == node:
                _zag(node, gp.left, grandnode=gp)
                _zig(node, gp, grandnode=gpp)
        #zag-zag
        if gp.right is not None:
            if gp.right.right == node:
                temp = gp.right
                _zag(gp.right, gp, grandnode=gpp)
                _zag(node, temp, grandnode=gpp)
        #zag-zig
            elif gp.right.left == node:
                _zig(node, gp.right, grandnode=gp)
                _zag(node, gp, grandnode=gpp)

        #loops splay operation until node is root
        return self.splayNode(root, node)
    
    def _parent(self, root, key):
        '''
        Returns parent of node with given key
        If parent is not found, None is returned
        '''
        if root.left is not None:
            if root.left.key == key: return root
            elif root.key > key: return self._parent(root.left, key)
        if root.right is not None:
            if root.right.key == key: return root
            elif root.key < key: return self._parent(root.right, key)
        return None

    def _inOrderGen(self, root):
        '''Returns inorder generator of all nodes'''
        if root.left: 
            for node in self._inOrderGen(root.left):
                yield node
        yield root
        if root.right:
            for node in self._inOrderGen(root.right):
                yield node
    
    def _inOrderReverseGen(self, root):
        '''Returns reverse inorder generator of all nodes'''
        if root.right:
            for node in self._inOrderReverseGen(root.right):
                yield node
        yield root
        if root.left:
            for node in self._inOrderReverseGen(root.left):
                yield node

    def _preOrderGen(self, root):
        '''Returns preorder generator of all nodes'''
        yield root
        if root.left:
            for node in self._preOrderGen(root.left):
                yield node
        if root.right:
            for node in self._preOrderGen(root.right):
                yield node
                
    def _postOrderGen(self, root):
        '''Returns postorder generator of all nodes'''
        if root.left:
            for node in self._postOrderGen(root.left):
                yield node
        if root.right:
            for node in self._postOrderGen(root.right):
                yield node
        yield root
        
    def _levelOrderGen(self, root, start=0):
        '''Returns levelorder generator of all nodes'''
        if start == 0: yield root
        if root.left: yield root.left
        if root.right: yield root.right
        if root.left:
            for node in self._levelOrderGen(root.left, start=1):
                yield node
        if root.right:
            for node in self._levelOrderGen(root.right, start=1):
                yield node
                
    def inOrder(self):
        '''Returns inorder list of all nodes'''
        return list(self._inOrderGen(self.root))
    
    def inOrderReverse(self):
        '''Returns reverse inorder list of all nodes'''
        return list(self._inOrderReverseGen(self.root))
    
    def preOrder(self):
        '''Returns preorder list of all nodes'''
        return list(self._preOrderGen(self.root))
    
    def postOrder(self):
        '''Returns postorder list of all nodes'''
        return list(self._postOrderGen(self.root))
                
    def levelOrder(self):
        '''Returns levelorder list of all nodes'''
        return list(self._levelOrderGen(self.root))        

In [4]:
class TitleTree(SplayBST):
    def search(self, key):
        node = self.searchNode(self.root, key)
        self.root = self.splayNode(self.root, node)
        categorizer = lambda x, k: [k] + list(zip(['score', 'rating', 'genre', 'box office', 'running time'], x))
        return categorizer(node.val, key)

In [5]:
class ScoreTree(SplayBST):
    def scoreprint(self, min=-1, max=101):
        for node in self._inOrderGen(self.root):
            if node.key > min and node.key < max: print(node) #note: min and max are not in the printed list

In [6]:
class BoxOfficeTree(SplayBST):
    def bocount(self, max=101, min=-1):
        count = 0
        for node in self._inOrderGen(self.root):
            if node.key > min and node.key < max: count += 1
        return count
    
    def boprint(self, min=-1, max=101):
        for node in self._inOrderGen(self.root):
            if node.key > min and node.key < max: print(node) #note: min and max are included in the printed list
                
    def highestboprint(self):
        def findRightMost(root):
            if root.right is not None: return findRightMost(root.right)
            else: return root
        print(findRightMost(self.root))

In [7]:
movies = open('movies.txt', 'r').readlines()
for i in range(len(movies)): movies[i] = movies[i][:-1].split("\t")

In [8]:
#Task 1 - addition of movie records
title = TitleTree(movies[0][0], movies[0][1:])
title2 = TitleTree(movies[0][0], movies[0][1:])
score = ScoreTree(float(movies[1][1]), movies[1][0])
boxoffice = BoxOfficeTree(float(movies[1][4]), movies[1][0])
for i in movies[1:]:
    title.add(i[0], i[1:])
    title2.add(i[0], i[1:])
    score.add(float(i[1]), (i[0]))
    boxoffice.add(float(i[4]), (i[0]))
for i in title:
    print(i)

('2 Fast 2 Furious', ['48.9', 'PG-13', 'action/adventure', '127.146', '107'])
('28 Days Later', ['78.2', 'R', 'horror', '45.065', '113'])
('A Guy Thing', ['39.5', 'PG-13', 'rom comedy', '15.545', '101'])
('A Man Apart', ['42.9', 'R', 'action/adventure', '26.248', '110'])
('A Mighty Wind', ['79.9', 'PG-13', 'comedy', '17.781', '91'])
('Agent Cody Banks', ['57.9', 'PG', 'action/adventure', '47.811', '102'])
('Alex & Emma', ['35.1', 'PG-13', 'rom comedy', '14.219', '96'])
('American Wedding', ['50.7', 'R', 'comedy', '104.441', '96'])
('Anger Management', ['62.6', 'PG-13', 'comedy', '134.404', '106'])
('Anything Else', ['63.3', 'R', 'rom comedy', '3.212', '108'])
('Bad Boys II', ['38.1', 'R', 'action/adventure', '138.397', '147'])
('Bad Santa', ['75.8', 'R', 'comedy', '59.523', '91'])
('Basic', ['43.6', 'R', 'suspense', '26.536', '98'])
('Beyond Borders', ['48.8', 'R', 'drama', '4.430', '127'])
('Biker Boyz', ['44.8', 'PG-13', 'action/adventure', '21.731', '110'])
('Boat Trip', ['29.2', 'R

In [9]:
#Task 2 - deletion of movie records
print('Before deletion:')
for i in title2:
    print(i)
for i in movies[1:]:
    title2.delete(i[0])
    
print('After deletion:')
print(title2)

Before deletion:
('2 Fast 2 Furious', ['48.9', 'PG-13', 'action/adventure', '127.146', '107'])
('28 Days Later', ['78.2', 'R', 'horror', '45.065', '113'])
('A Guy Thing', ['39.5', 'PG-13', 'rom comedy', '15.545', '101'])
('A Man Apart', ['42.9', 'R', 'action/adventure', '26.248', '110'])
('A Mighty Wind', ['79.9', 'PG-13', 'comedy', '17.781', '91'])
('Agent Cody Banks', ['57.9', 'PG', 'action/adventure', '47.811', '102'])
('Alex & Emma', ['35.1', 'PG-13', 'rom comedy', '14.219', '96'])
('American Wedding', ['50.7', 'R', 'comedy', '104.441', '96'])
('Anger Management', ['62.6', 'PG-13', 'comedy', '134.404', '106'])
('Anything Else', ['63.3', 'R', 'rom comedy', '3.212', '108'])
('Bad Boys II', ['38.1', 'R', 'action/adventure', '138.397', '147'])
('Bad Santa', ['75.8', 'R', 'comedy', '59.523', '91'])
('Basic', ['43.6', 'R', 'suspense', '26.536', '98'])
('Beyond Borders', ['48.8', 'R', 'drama', '4.430', '127'])
('Biker Boyz', ['44.8', 'PG-13', 'action/adventure', '21.731', '110'])
('Boat T

After deletion:
[('title', ['score', 'rating', 'genre', 'box office', 'running time'])]


In [10]:
#Task 3 - searching titles
print(title.search('2 Fast 2 Furious'))
print('')
print(title.search('28 Days Later'))
print('')
print(title.search('big fish'))

['2 Fast 2 Furious', ('score', '48.9'), ('rating', 'PG-13'), ('genre', 'action/adventure'), ('box office', '127.146'), ('running time', '107')]

['28 Days Later', ('score', '78.2'), ('rating', 'R'), ('genre', 'horror'), ('box office', '45.065'), ('running time', '113')]

['big fish', ('score', '78.0'), ('rating', 'PG-13'), ('genre', 'drama'), ('box office', '65.151'), ('running time', '125')]


In [11]:
#Task 4 - count number of titles with less than y million dollars in box office
print(boxoffice.bocount(30)) #in this case, y is 30

49


In [12]:
#Task 5 - display all titles with box office within range [a, b]
boxoffice.boprint(70, 90) #does not include 70 and 90 mil.

(74.32, 'The Haunted Mansion')
(74.663, 'Old School')
(79.207, 'Mystic River')
(80.168, 'The Texas Chainsaw Massacre')
(81.239, 'The School of Rock')
(82.217, 'Freddy vs. Jason')
(84.874, 'Brother Bear')
(89.921, 'Legally Blonde 2')


In [13]:
#Task 6 - print the movie with the highest box office
boxoffice.highestboprint()

(361.119, 'The Lord of the Rings III')


In [14]:
#Task 7 - print all titles by ascending score
score.scoreprint()

(29.2, 'Boat Trip')
(29.3, 'Gigli')
(29.5, 'Dumb and Dumberer')
(29.8, 'The Order')
(30.4, 'Kangaroo Jack')
(31.1, 'Darkness Falls')
(32.6, 'Wrong Turn')
(33.2, "My Boss's Daughter")
(34.8, 'From Justin to Kelly')
(35.0, 'Grind')
(35.1, 'Alex & Emma')
(36.2, 'Marci X')
(36.3, 'House of 1000 Corpses')
(37.8, "Dr. Seuss' The Cat in the Hat")
(37.9, 'National Security')
(38.1, 'Bad Boys II')
(38.5, 'Uptown Girls')
(39.5, 'A Guy Thing')
(40.5, 'Daddy Day Care')
(40.8, 'The Real Cancun')
(41.5, 'Just Married')
(42.0, 'Gods and Generals')
(42.2, 'The Texas Chainsaw Massacre')
(42.5, 'Dreamcatcher')
(42.9, 'Timeline')
(43.2, 'The League of Extraordinary Gentlemen')
(43.6, 'Basic')
(43.7, 'View from the Top')
(44.2, 'Jeepers Creepers 2')
(44.8, 'The In-Laws')
(45.3, 'The Haunted Mansion')
(45.4, "Love Don't Cost a Thing")
(45.8, 'Cradle 2 the Grave')
(46.1, 'Final Destination 2')
(46.2, 'Underworld')
(46.9, 'The Life of David Gale')
(47.3, 'Freddy vs. Jason')
(47.9, 'Hollywood Homicide')
(48.1

In [15]:
#Task 8 - print all titles with score higher than x
score.scoreprint(90) #in this case, x is 90

(91.2, 'Finding Nemo')
(92.2, 'The Lord of the Rings III')
(94.6, 'Lost in Translation')
